# Bestimmung von Wortarten und Morphologie

Wortarten spielen nicht umsonst eine sehr große Rolle in der Sprache. Aber auch die Art und Weise, wie Wörter verwendet werden, trägt erheblich zur Kommunikation bei - du kennst vielleicht noch den *Ich-Erzähler* aus der Schule.

Zum Glück können alle diese Analysen auch von `spacy` durchgeführt werden

## Basisdaten

Wie gewohnt beginnst du wieder mit den beiden Paragraphen aus dem Neujahrs-Artikel:

In [ ]:
p1 = "Doch das Ende des Jahres 2020 birgt auch Hoffnung, dass durch die Vakzinen \
gegen Covid-19 wieder Normalität einkehre – wie immer die auch aussehen mag \
– und wir uns um anderes Dringliches kümmern oder einfach entspannen \
können. Und dass durch den im Januar anstehenden Bewohnerwechsel im \
Weißen Haus zu Washington D.C. das offizielle Herumgetrumpel auf dem \
gesunden Menschenverstand ein Ende finden möge."

p2 = "Wir, das gesamte Team von heise online und die Redaktionen von c't, iX, \
Technology Review, Mac & i, c't Digitale Fotografie, Make:, Techstage und \
Telepolis sowie heise Security, heise Developer und heise Autos wünschen Ihnen \
ein friedliches und freudvolles Jahr 2021. Wir wünschen Ihnen, dass Sie nicht \
vergeblich hoffen und dass Ihre Vorsätze erfüllt werden, auf dass Sie gesund \
bleiben oder genesen."

## Analyse

Mithilfe von `spacy` analysierst du die Texte:

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import spacy

nlp = spacy.load("de_core_news_lg")
d1 = nlp(p1)
d2 = nlp(p2)

## Ausgabe

Der *Tag* ist ziemlich präzise, außerdem lässt du dir noch *POS* und die *Morphologie* ausgeben:

In [ ]:
import pandas as pd

pd.set_option("max.rows", None)
pd.set_option('display.max_colwidth', None)

def tag_morph(d):
    res = []
    for token in d:
        res.append([token.text, token.tag_, token.pos_, str(token.morph)])

    return pd.DataFrame(res, columns=["Text", "Tag", "POS", "Morphologie"])

In [ ]:
tm1 = tag_morph(d1)
tm1

Die Morphologie ist relativ neu in `spacy` integriert und steht ab Version `3.0` zur Verfügung. Die darin enthaltenen Informationen kannst du z.B. nutzen, um eine bestimmte *Textart* zu erkennen (und später zu klassifizieren). Auch der Schreibstil ist implizit in der Morphologie enthalten, weil über `Mood` und `VerbForm` viel Information transportiert wird.

## POS und Tag

`POS` und `Tag` drücken jeweils *Wortarten* aus, `Tag` ist dabei allerdings genauer. An Beispielen siehst du die Unterschiede gut:

In [ ]:
# Substantive
tm1[(tm1["POS"] == "NOUN") | (tm1["POS"] == "PROPN")]["Tag"].unique()

In [ ]:
# Adjektive
tm1[(tm1["POS"] == "ADJ") | (tm1["POS"] == "ADV")]["Tag"].unique()

In [ ]:
# Verben
tm1[(tm1["POS"] == "VERB") | (tm1["POS"] == "AUX")]["Tag"].unique()

Im nächsten Schritt zerlegst du nun den Text in *Substantive*, *Adjektive* und *Verben*, von denen du dir nur die Grundform merkst. In `nav` speicherst du diese Wortarten in der richtigen Reihenfolge ab. Da du dies häufiger durchführen musst, bietet es sich an, das in einer Funktion zu kapseln:

In [ ]:
def word_types(doc):
    nouns = []
    adjectives = []
    verbs = []
    lemmas = []
    nav = []
    for token in doc:
        lemmas.append(token.lemma_)
        # adjective
        if token.pos_ == "ADJ" or token.pos_ == "ADV":
            adjectives.append(token.lemma_)
            nav.append(token.lemma_)
        # noun
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            nouns.append(token.lemma_)
            nav.append(token.lemma_)
        # verb
        if token.pos_ == "VERB":
            verbs.append(token.lemma_)
            nav.append(token.lemma_)
            
    return (nouns, adjectives, verbs, nav, lemmas)

In [ ]:
(nouns, adjectives, verbs, nav, lemmas) = word_types(d1)
print(f'Substantive: {"|".join(nouns)}')
print(f'Adjektive: {"|".join(adjectives)}')
print(f'Verben: {"|".join(verbs)}')
print(f'Kombinationen: {"|".join(nav)}')
print(f'Lemmas: {"|".join(lemmas)}')

Wenn du dir die Ergebnisse ansiehst, wirst du feststellen, dass die Substantive und Verben sehr gut erkannt wurden. Einige Adjektive sind in Wahrheit allerdings Substantive, d.h. damit hat `spacy` ab und zu noch Schwierigkeiten. Wenn du magst, tausche das Modell aus und übrprüfe, ob sich dadurch Verbesserungen ergeben.